In [ ]:
#!apt install cmake

In [ ]:
!git clone https://github.com/cloner174/mr.DGA.git

In [ ]:
%cd mr.DGA

In [1]:
import pandas as pd
import os

In [2]:
# path to data

data_path = 'data/input/'

In [3]:
# Load data

name_of_batches = os.listdir(data_path)

data_batches_path = []
for any_batch_name in name_of_batches :
    
    path_to_this_batch = os.path.join(data_path, any_batch_name)
    data_batches_path.append(path_to_this_batch)

data_batches_path.sort()

data_batches = []
for path_to_this_batch in data_batches_path :
    
    temp_batch = pd.read_csv(path_to_this_batch, compression='zip')
    data_batches.append(temp_batch)

data = pd.concat(data_batches, axis=0, ignore_index=True)
#data = pd.read_csv(data_path)

In [4]:
data.head(), '\n', data.shape

(   emotion                                             pixels
 0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
 1        0  151 150 147 155 148 133 111 140 170 174 182 15...
 2        2  231 212 156 164 174 138 161 173 182 200 106 38...
 3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
 4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,
 '\n',
 (35887, 2))

In [ ]:
%pip install imutils opencv-python dlib opencv-python-headless pillow matplotlib mediapipe face_recognition

In [5]:
import cv2
import face_recognition
import numpy as np

In [6]:
def strings_with_number_inside( data,
                                    column_range_starts : int = 0,
                                    column_range_ends : int = None,
                                    seperator = None):
        start = column_range_starts
        end = int( data.shape[1] ) if column_range_ends is None else column_range_ends
        new_data = {}
        if isinstance( data , pd.DataFrame ) :
            for i in range( data.shape[0] ) :
                temp_new_data = []
                for j in range( start, end ) :
                    cells_real_values = np.array(data.iloc[i, j].split(sep = seperator), dtype='uint8')
                    temp_new_data.append(cells_real_values)
                new_data[i] = temp_new_data

            return new_data
        else:
            if isinstance( data , np.array ) :
                pass
            else:
                raise TypeError( " data just could be  PandasDataFrame  or  NumpyArray  ! ")

In [7]:
# path to save images

face_directory = 'data/output/pic/face'
non_face_directory = 'data/output/pic/not_face/'

In [8]:
# each row corresponds to an image
image_size = (48, 48)  #dimensions of image

In [ ]:
#   #       # #   #   #       # ##   #       # ##   #       # ##   #       # ##   #       # ##   #       # #

In [9]:
print('path to faces folder: ' ,face_directory,'\n', 'path to non face folder : ' ,non_face_directory)

path to faces folder:  data/output/pic/face 
 path to non face folder :  data/output/pic/not_face/


In [10]:
#path_to_your_haarcascade_face_model_from_opencv

path_to_haarcascade_face_model = 'models/openCV/haarcascade_frontalface_default.xml'

In [11]:
face_cascade = cv2.CascadeClassifier(path_to_haarcascade_face_model)

In [ ]:
data_pix = strings_with_number_inside(data, column_range_starts = 1)

In [ ]:
os.makedirs(face_directory, exist_ok=True)
os.makedirs(non_face_directory, exist_ok=True)

# loop to process images
for index, row in data_pix.items():
    image_array = np.array(row, dtype=np.uint8).reshape(image_size[0], image_size[1])
    faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.1, minNeighbors=5)
    face_recognition, needs image in rgb even if its gray
    if len(faces) == 0:
      cv2.imwrite(os.path.join(non_face_directory, f'non_face_{index}.jpg'), image_array)
    else:
        image_rgb = np.stack((image_array,) * 3, axis=-1)
        face_locations = face_recognition.face_locations(image_rgb, model='cnn')  #CNN model
        for face_location in face_recognition:
          top, right, bottom, left = face_location
          face_image = image_array[top:bottom, left:right]  #cropping
          cv2.imwrite(os.path.join(face_directory, f'face_{index}.jpg'), face_image)
        if len(face_locations) == 0:
          cv2.imwrite(os.path.join(non_face_directory, f'non_face_{index}.jpg'), image_array)

print("Face cropping and saving completed.")

In [14]:
face_folder = os.listdir(face_directory)
non_face_folder = os.listdir(non_face_directory)


print('Number of pictures with good face that have been detected by model : ',len(face_folder))
print('Number of pictures without any face detected :' , len(non_face_folder))

Number of pictures with good face that have been detected by model :  4426
Number of pictures without any face detected : 14870


In [15]:
len(face_folder)+ len(non_face_folder) == data.shape[0]

False

In [16]:
import re
images_emo_map = {}
for any_ in face_folder :
  temp_path = re.search('\d+', any_)
  temp_num = int(temp_path.group())
  images_emo_map[any_] = {}
  images_emo_map[any_]['path'] = os.path.join(face_directory, any_)
  images_emo_map[any_]['emotion'] = data.loc[temp_num, 'emotion']
  images_emo_map[any_]['iloc'] = temp_num


KeyError: 7082

In [17]:
len(images_emo_map.keys()) == len(face_folder)

False

In [18]:
import subprocess
from main import Utilities

In [19]:
base_image_dir = 'data/output/pic'
labels = data['emotion'].unique()

In [20]:
# directories for each label
directories = Utilities.create_directories(base_image_dir, labels)

In [21]:
directories

{0: 'data/output/pic/0',
 6: 'data/output/pic/6',
 3: 'data/output/pic/3',
 5: 'data/output/pic/5',
 4: 'data/output/pic/4',
 2: 'data/output/pic/2',
 1: 'data/output/pic/1'}

In [22]:
# Process each image in the dataframe
for index, row in directories.items():
   for images_name in face_folder :
     if images_emo_map[images_name]['emotion'] == index :
      subprocess.call(['mv', images_emo_map[images_name]['path'], row])

KeyError: 'emotion'

In [23]:
c = 0
for indx, any_ in directories.items():
  temp_listdir = os.listdir(any_)
  print(f'the number of valid images with emotion {indx} is :', len(temp_listdir))
  c += len(temp_listdir)
print('\n --> ',c)
print( '-->' ,c == len(face_folder))

the number of valid images with emotion 0 is : 0
the number of valid images with emotion 6 is : 0
the number of valid images with emotion 3 is : 0
the number of valid images with emotion 5 is : 0
the number of valid images with emotion 4 is : 0
the number of valid images with emotion 2 is : 0
the number of valid images with emotion 1 is : 0

 -->  0
--> False


In [24]:
valid_rows = []
for indx in images_emo_map.keys():

  valid_rows.append(images_emo_map[indx]['iloc'])
len(valid_rows)

KeyError: 'iloc'

In [ ]:
data_valid = data.iloc[valid_rows,:]

data_valid.shape

(8194, 2)

In [ ]:
base_image_csv = 'data/output/csv'
os.makedirs(base_image_csv, exist_ok=True)
labels = data['emotion'].unique()

In [ ]:
for label in labels:
    subset = data_valid[data_valid['emotion'] == label]
    print(f'\n Number of rows of valid data with emotion {label} label : ', subset.shape[0])
    file_path = os.path.join(base_image_csv, f'data-valid-label{label}.csv')
    subset.to_csv(file_path, index=False)
    print(f"Data for label {label} saved to {file_path}")
